In [37]:
from pathlib import Path

import numpy as np
import pandas as pd

from apo_holo_structure_stats.paper_repl.main import get_paper_apo_holo_dataframe

PIPELINE_OUTPUT_FILE = Path('../../o_isoform_paper2.json')

paper_pairs = get_paper_apo_holo_dataframe()
paper_pairs


,domain_count,ligand_codes,apo_chain_id,holo_chain_id,apo_pdb_code,holo_pdb_code
0,1,SUC;SUC;,R,P,1a0s,1a0t
1,1,SES;,A,A,1a3y,1dzj
2,1,BEZ;,B,A,1a7u,1a8u
3,2,NGA;NGA;,_,A,1a8d,1d0h
4,2,OXL;,_,A,1a8f,1ryo
...,...,...,...,...,...,...
516,1,BHC;,D,A,4pgm,1bq4
517,1,5AS;,_,A,6rhn,1rzy
518,2,NAG-MAN;,_,A,6taa,2guy
519,2,GOL;GOL;GOL;,D,B,7req,1req


Podvybrat pouze chainy struktur, tak jako v paperu.

In [38]:


output = pd.read_json(PIPELINE_OUTPUT_FILE)

output = output.dropna()
# vyres to tak: pro vsechny podtrzitka najdi v outputu spravny struktury a ma byt prave 1 myslim
for i, row in enumerate(paper_pairs.itertuples()):
    row = row._asdict()  # convert to dict, so we can easily index columns with strings

    for apo_or_holo in ('apo', 'holo'):
        if row[f'{apo_or_holo}_chain_id'] != '_':
            continue

        hits = output[output.pdb_code == row[f'{apo_or_holo}_pdb_code']]
        if len(hits) != 1:
            print('error', row[f'{apo_or_holo}_pdb_code'])
            raise RuntimeError()

        # change '_' into actual chain id, so we can do merges with pd
        paper_pairs[f'{apo_or_holo}_chain_id'].iat[i] = hits['chain_id'].iat[0]

# todo takhlle nejde mergovat, z nejakyho duvodu mam 2 dupes
output = output.merge(paper_pairs, how='left', left_on=['pdb_code', 'chain_id'], right_on=['apo_pdb_code', 'apo_chain_id'])
output = output.merge(paper_pairs, how='left', left_on=['pdb_code', 'chain_id'], right_on=['holo_pdb_code', 'holo_chain_id'])
output = output.dropna(subset=['apo_pdb_code_x', 'apo_pdb_code_y'], how='all')
print('apo', np.sum(output.apo_pdb_code_y.isna()))
print('holo', np.sum(output.apo_pdb_code_x.isna()))
output.insert(loc=3, column='is_holo_paper', value=output.apo_pdb_code_x.isna())
group_sizes = output.groupby('isoform').count()

# dat isholo do sklearn metrics
from sklearn.metrics import classification_report

print(classification_report(output.is_holo_paper, output.is_holo))
output[output.is_holo_paper != output.is_holo]


apo 523
holo 528
              precision    recall  f1-score   support

       False       0.95      0.96      0.95       523
        True       0.96      0.95      0.95       528

    accuracy                           0.95      1051
   macro avg       0.95      0.95      0.95      1051
weighted avg       0.95      0.95      0.95      1051



,pdb_code,path,chain_id,is_holo_paper,is_holo,isoform,domain_count_x,ligand_codes_x,apo_chain_id_x,holo_chain_id_x,apo_pdb_code_x,holo_pdb_code_x,domain_count_y,ligand_codes_y,apo_chain_id_y,holo_chain_id_y,apo_pdb_code_y,holo_pdb_code_y
79,1c60,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P00720,NaN,NaN,NaN,NaN,NaN,NaN,2.0,BME;,A,A,1c62,1c60
115,1d1q,apo_holo_structure_stats/paper_repl/pdb_struct...,B,False,True,P40347,1.0,ADE;,B,B,1d1q,1d2a,NaN,NaN,NaN,NaN,NaN,NaN
204,1f4v,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P0AE67,NaN,NaN,NaN,NaN,NaN,NaN,1.0,MET-GLY-ASP-SER-ILE-LEU-SER-GLN-ALA-GLU-ILE-AS...,A,A,1eay,1f4v
211,1ehd,apo_holo_structure_stats/paper_repl/pdb_struct...,A,False,True,Q9S7B3,2.0,NAG;NAG;,A,A,1ehd,1ehh,NaN,NaN,NaN,NaN,NaN,NaN
218,1eoa,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P20371,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CYN;,A,A,1eo2,1eoa
219,1eoa,apo_holo_structure_stats/paper_repl/pdb_struct...,B,True,False,P20372,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CYN;,B,B,1eo2,1eoa
222,1eoa,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P20371,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CYN;,A,A,1eo2,1eoa
223,1eoa,apo_holo_structure_stats/paper_repl/pdb_struct...,B,True,False,P20372,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CYN;,B,B,1eo2,1eoa
264,2et1,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P45850,NaN,NaN,NaN,NaN,NaN,NaN,2.0,GLV;,A,A,1fi2,2et1
397,1xz1,apo_holo_structure_stats/paper_repl/pdb_struct...,A,True,False,P02791,NaN,NaN,NaN,NaN,NaN,NaN,1.0,HLT;,A,A,1gwg,1xz1


Porovnat assignment apo, holo jednotlive (po strukturach)
- FP, FN, TP, TN

Porovnat skupiny (vsechny vel. 2 snad)

Porovnat pocet paru, ktery nam vysel stejne, (zbytek budou pary, ktere se nevytvorily, bud oba apo nebo oba holo -> počty)